In [35]:
!mkdir -p ./data/corpus/train

# We are now focusing on lenta.ru news dataset

In [6]:
URL: str = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/0.1/news_lenta.csv"

In [83]:
from tqdm.auto import tqdm
import requests
from flair.data import Dictionary
from collections import Counter
import pickle

import glob

In [8]:
def download(file_name, source_url):
    CHUNK = 16 * 1024

    r = requests.get(source_url, stream=True)
    total_length = int(r.headers.get('content-length', 0))

    with open(file_name, 'wb') as f:
        pbar = tqdm(total=total_length, unit='B', unit_scale=True)
        for chunk in r.iter_content(chunk_size=CHUNK):
            if chunk:  # filter out keep-alive new chunks
                pbar.update(len(chunk))
                f.write(chunk)
        f.close()

In [ ]:
download("./data/news_lenta.csv", URL)

In [10]:
!ls ./data/

news_lenta.csv


In [11]:
import pandas as pd

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split

In [13]:
data = pd.read_csv("./data/news_lenta.csv")

In [15]:
data.head(1)

,tags,text,title,topic,url
0,Общество,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,https://lenta.ru/news/2018/07/16/su57/


In [20]:
data = data.drop(['tags', 'title', 'topic', 'url'], axis=1)

In [21]:
data.head(1)

,text
0,Миллиардер Илон Маск в резкой форме ответил бр...


In [40]:
train_and_valid, test = train_test_split(data.text, test_size=0.05, random_state=42)

In [41]:
print(f"Train\Dev size: {len(train_and_valid)}, Test size: {len(test)}")

Train\Dev size: 664758, Test size: 34988


In [42]:
train, valid = train_test_split(train_and_valid, test_size=0.05, random_state=42)

In [43]:
print(f"Train size: {len(train)}, Valid size: {len(valid)}, Test size: {len(test)}")

Train size: 631520, Valid size: 33238, Test size: 34988


In [78]:
N_parts = 80

In [79]:
parts_sizes = len(train)/N_parts
print(parts_sizes)

7894.0


In [46]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [47]:
def write_to_file(sentences, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for sentence in tqdm(sentences):
            f.write(f"{sentence}\n")

In [48]:
write_to_file(test.values, './data/corpus/test.txt')

In [49]:
write_to_file(valid.values, "./data/corpus/valid.txt")

In [81]:
for i, chunk in enumerate(chunks(train, int(parts_sizes))):
    write_to_file(chunk, f"./data/corpus/train/train_split_{i+1}")

In [84]:
# make an empty character dictionary
char_dictionary: Dictionary = Dictionary()

# counter object
counter = Counter()

processed = 0

files = glob.glob('./data/corpus/train/*')

#print(files)
for file in tqdm(files):
    #print(file)

    with open(file, 'r', encoding='utf-8') as f:
        tokens = 0
        for line in f:

            processed += 1            
            chars = list(line)
            tokens += len(chars)

            # Add chars to the dictionary
            counter.update(chars)

            # comment this line in to speed things up (if the corpus is too large)
            # if tokens > 50000000: break

    # break

total_count = 0
for letter, count in counter.most_common():
    total_count += count

print(total_count)
print(processed)

sum = 0
idx = 0
for letter, count in counter.most_common():
    sum += count
    percentile = (sum / total_count)

    # comment this line in to use only top X percentile of chars, otherwise filter later
    # if percentile < 0.00001: break

    char_dictionary.add_item(letter)
    idx += 1
    print('%d\t%s\t%7d\t%7d\t%f' % (idx, letter, count, sum, percentile))

print(char_dictionary.item2idx)

with open('./data/dictionaty.pckl', 'wb') as f:
    mappings = {
        'idx2item': char_dictionary.idx2item,
        'item2idx': char_dictionary.item2idx
    }
    pickle.dump(mappings, f)

840284041
669534
1	 	112872662	112872662	0.134327
2	о	71643756	184516418	0.219588
3	а	55244822	239761240	0.285334
4	е	55175886	294937126	0.350997
5	и	53182627	348119753	0.414288
6	н	43573334	391693087	0.466144
7	т	41308131	433001218	0.515303
8	р	36629176	469630394	0.558895
9	с	36404106	506034500	0.602218
10	в	30452165	536486665	0.638459
11	л	28322638	564809303	0.672165
12	к	21423757	586233060	0.697661
13	д	19551120	605784180	0.720928
14	м	18970750	624754930	0.743504
15	п	18772037	643526967	0.765845
16	у	15270678	658797645	0.784018
17	я	12160548	670958193	0.798490
18	ы	11548091	682506284	0.812233
19	г	10686014	693192298	0.824950
20	б	10589708	703782006	0.837553
21	з	10500115	714282121	0.850048
22	,	9291962	723574083	0.861107
23	ь	8581478	732155561	0.871319
24	й	8304644	740460205	0.881202
25	ч	7803636	748263841	0.890489
26	.	7175327	755439168	0.899028
27	х	5386035	760825203	0.905438
28	ж	5221663	766046866	0.911652
29	ц	4212427	770259293	0.916665
30	ю	3693701	773952994	0.921061
31	ш	35396